In [ ]:
import os
from pathlib import Path


current_team = "dev_team"
dataset_dir = Path(f"../data/remote_dry_run_2024-04-26/{current_team}")

sample_id = "2024-04-23_10-20-07-968516"

sample_dir = dataset_dir / f"sample_{sample_id}"
grasps_dir = dataset_dir / f"grasps_{sample_id}" # TODO throughout this notebook add grasp.json files 

os.path.exists(grasps_dir)

In [ ]:
import time
from loguru import logger
from airo_dataset_tools.data_parsers.pose import Pose


failed_files = set()

while True:
    grasp_files = set(os.listdir(grasps_dir))
    files_to_consider = grasp_files - failed_files

    if not files_to_consider:
        logger.info("No grasp poses received yet, waiting...")
        time.sleep(1.0)
        continue

    grasp_files = sorted(list(files_to_consider), reverse=True)
    latest_file = grasp_files[0]

    logger.info(f"Trying grasp pose from: {latest_file}")

    filepath = os.path.join(grasps_dir, latest_file)

    try:
        with open(filepath, "r") as f:
            grasp_pose = Pose.model_validate_json(f.read()).as_homogeneous_matrix()
    except Exception as e:
        failed_files.add(latest_file)
        logger.warning(f"Cannot load {latest_file}: {e}")
        time.sleep(0.0001)
        continue

    logger.success(f"Loaded grasp pose successfully from: {latest_file}")
    break